In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from scipy.io import arff
import numpy as np
import pandas as pd

# Load the ARFF file
data, meta = arff.loadarff('dna.arff')

# Convert structured array to DataFrame
df = pd.DataFrame(data)

# Decode byte strings (if necessary)
for col in df.select_dtypes([np.object_]):
    df[col] = df[col].str.decode('utf-8')
# Separate features and target variable
y = df.iloc[:, -1]  # Labels are in the first column
X = df.iloc[:, :-1]  # Features start from index 1

# Encode class labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['number']).columns.tolist()

# Define preprocessing steps
preprocessor = ColumnTransformer([
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'), categorical_cols),  # Drop first to avoid extra column
    ('scaler', MinMaxScaler(), numerical_cols)
])


# Apply transformations
X = preprocessor.fit_transform(X)

# Split data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

if hasattr(X_train, "toarray"):  # Check if X_train is a sparse matrix
    X_train = X_train.toarray()
    X_val = X_val.toarray()
    X_test = X_test.toarray()

# Print dataset shapes
print("Train set size:", X_train.shape)
print("Validation set size:", X_val.shape)
print("Test set size:", X_test.shape)

# Save datasets as NumPy arrays
np.save("train_x.npy", X_train)
np.save("val_x.npy", X_val)
np.save("xtest.npy", X_test)
np.save("ytrain.npy", y_train)
np.save("yval.npy", y_val)
np.save("ytest.npy", y_test)


Train set size: (2038, 180)
Validation set size: (510, 180)
Test set size: (638, 180)
